**Goal is to use smoltalk dataset containing chat templates and fine tune the deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B model with LoRA config and quantization using BitsAndBytesConfig**

*Note: Every input, parameters passed was refined/chosen to complete the task using google collab free tier GPU allocation**

**Install necessary libraries**

In [ ]:
!pip install -q transformers peft accelerate bitsandbytes datasets trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Import libraries and loading model with quantization**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Model checkpoint
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# 4-bit quantization config
quant_config = BitsAndBytesConfig(load_in_4bit=True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

# Check if using GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Using device: cuda


**Load the dataset and select 1000 samples**

In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("HuggingFaceTB/smoltalk",'smol-summarize')

# Subsample to 1,000 samples
small_dataset = dataset["train"].select(range(1000))

print(small_dataset)

README.md:   0%|          | 0.00/9.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/119M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/6.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/96356 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5072 [00:00<?, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 1000
})


**Defining LoRA and applying to model**

In [ ]:
from peft import LoraConfig, get_peft_model

# LoRA configuration
lora_config = LoraConfig(
    r=16,                    # Rank
    lora_alpha=32,           # Alpha scaling
    lora_dropout=0.1,        # Dropout for stability
    bias="none",
    task_type="CAUSAL_LM"    # Text generation model
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()


trainable params: 2,179,072 || all params: 1,779,267,072 || trainable%: 0.1225


**Training the model with 3 epochs**

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer  # Hugging Face's Trainer for LoRA

# Define training arguments
training_args = TrainingArguments(
    output_dir="./deepseek-lora",
    per_device_train_batch_size=1,  # Reduce if out of memory
    gradient_accumulation_steps=16,  # Helps when batch size is small
    num_train_epochs=3,             # Number of epochs
    save_steps=100,                 # Save every 100 steps
    logging_steps=50,               # Log progress
    learning_rate=2e-4,             # LoRA training needs higher LR
    fp16=True,                      # Mixed precision for speed
    optim="paged_adamw_32bit",      # Optimized AdamW optimizer
    push_to_hub=False               # Set True to save model to Hugging Face Hub
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=small_dataset,  # Our 1,000 sample dataset
    peft_config=lora_config,  # LoRA configuration
    tokenizer=tokenizer,
)

# Start training
trainer.train()


<ipython-input-6-bc29773bb8fa>:19: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: n-harshavardhana1992 (n-harshavardhana1992-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
50,2.852000
100,2.557700
150,2.443000


TrainOutput(global_step=186, training_loss=2.58345208116757, metrics={'train_runtime': 1088.0515, 'train_samples_per_second': 2.757, 'train_steps_per_second': 0.171, 'total_flos': 1.112333665660416e+16, 'train_loss': 2.58345208116757})

**Using a sample to test the inference**

In [ ]:
# Prepare input for chat template
sample_input = small_dataset[5]

# Ensure correct chat format
formatted_input = tokenizer.apply_chat_template(
    sample_input["messages"],
    tokenize=False,  # Keep as raw text
    add_generation_prompt=True  # Ensures the assistant generates a response
)

# Tokenize input
inputs = tokenizer(formatted_input, return_tensors="pt").to(model.device)

# Run inference
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=500)

# Decode and display the model's response
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print("\n🔍 **Model Prediction:**")
print(decoded_output)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



🔍 **Model Prediction:**
Provide a concise, objective summary of the input text in up to three sentences, focusing on key actions and intentions without using second or third person pronouns.<｜User｜>By . Mark Prigg . It is certainly a unique sight, and one that could, it is claimed, save thousands of lives. A bicycle with a giant set of light up cat's whiskers if the latest gadget aiming to make cyclist's lives safer. They can be used to warn other road users a cyclist is approaching, let drivers know exactly how much space is needed, and even twitch to indicate a turn. Scroll down for video . The life-saving whiskers: The strands of smart material can extend to mark space, and twitch when the user wants to turn . The four whiskers can change their configuration, allowing the cyclists to signal their intentions. Controlled by a button on the handlebars, the four long whiskers on the Vibrasee can flick fully outward, discouraging a car from squeezing by. Reflective stripes make each whi

Based on the model prediction, model is doing a pretty good job even though it was trained for 3 epochs with subsamples of 1000, it is providing a concise summary based on the input instructions.

**Input passed**

In [ ]:
small_dataset[5]

{'messages': [{'content': 'Provide a concise, objective summary of the input text in up to three sentences, focusing on key actions and intentions without using second or third person pronouns.',
   'role': 'system'},
  {'content': "By . Mark Prigg . It is certainly a unique sight, and one that could, it is claimed, save thousands of lives. A bicycle with a giant set of light up cat's whiskers if the latest gadget aiming to make cyclist's lives safer. They can be used to warn other road users a cyclist is approaching, let drivers know exactly how much space is needed, and even twitch to indicate a turn. Scroll down for video . The life-saving whiskers: The strands of smart material can extend to mark space, and twitch when the user wants to turn . The four whiskers can change their configuration, allowing the cyclists to signal their intentions. Controlled by a button on the handlebars, the four long whiskers on the Vibrasee can flick fully outward, discouraging a car from squeezing by